In [11]:
import tqdm 
class DraftState:
    def __init__(self, rosters, turns, freeagents, playerjm=None):
        self.rosters = rosters
        self.turns = turns
        self.freeagents = freeagents
        self.playerJustMoved = playerjm
        
class NflPlayer:
    def __init__(self, name, position, points):
        self.name = name
        self.position = position
        self.points = points
        
    def __repr__(self):
        return "|".join([self.name, self.position, str(self.points)])

    
import numpy as np

def GetResult(self, playerjm):
    """ Get the game result from the viewpoint of playerjm.
    """
    if playerjm is None: return 0
    
    pos_wgts = {
        ("QB"): [1, 1],
        ("WR"): [1, 1, 1, 1],
        ("RB"): [1, 1, 1, 1],
        ("TE"): [1, 1],
        ("RB", "WR", 'TE'): [1, 1, 1]
    }


    result = 0
    # map the drafted players to the weights
    for p in self.rosters[playerjm]:
        max_wgt, _, max_pos, old_wgts = max(
            ((wgts[0], -len(lineup_pos), lineup_pos, wgts) for lineup_pos, wgts in pos_wgts.items()
                if p.position in lineup_pos),
            default=(0, 0, (), []))
        if max_wgt > 0:
            result += max_wgt * p.points
            old_wgts.pop(0)
            if not old_wgts:
                pos_wgts.pop(max_pos)
                
    # map the remaining weights to the top three free agents
    for pos, wgts in pos_wgts.items():
        result += np.mean([p.points for p in self.freeagents if p.position in pos][:3]) * sum(wgts)
        
    return result
        
DraftState.GetResult = GetResult


def GetMoves(self):
    """ Get all possible moves from this state.
    """
    pos_max = {"QB": 2, "WR": 6, "RB": 6, "TE": 2}

    if len(self.turns) == 0: return []

    roster_positions = np.array([p.position for p in self.rosters[self.turns[0]]], dtype=str)
    moves = [pos for pos, max_ in pos_max.items() if np.sum(roster_positions == pos) < max_]
    return moves

DraftState.GetMoves = GetMoves

In [12]:

def DoMove(self, move):
    """ Update a state by carrying out the given move.
        Must update playerJustMoved.
    """
    player = next(p for p in self.freeagents if p.position == move)
    self.freeagents.remove(player)
    rosterId = self.turns.pop(0)
    self.rosters[rosterId].append(player)
    self.playerJustMoved = rosterId
    
DraftState.DoMove = DoMove

In [13]:

def Clone(self):
    """ Create a deep clone of this game state.
    """
    rosters = list(map(lambda r: r[:], self.rosters))
    st = DraftState(rosters, self.turns[:], self.freeagents[:],
            self.playerJustMoved)
    return st

DraftState.Clone = Clone

In [23]:

# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai

from math import *
import random

class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        UCTK = 200
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result


def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
    """

    rootnode = Node(state = rootstate)

    for i in tqdm.tqdm(range(itermax)):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

In [24]:
import pandas as pd
nfl_players = pd.read_csv('all_predictions.csv')

In [25]:
nfl_players = nfl_players[['Name','FantPos', 'prediction']]
# nfl_players['point'] = nfl_players.point / 16

In [26]:
nfl_players.sort_values('prediction')

,Name,FantPos,prediction
542,Shane Smith,0,15.262641
207,Dontrell Hilliard,RB,43.224883
90,Cameron Batson,WR,43.789774
157,Darrel Williams,RB,44.692254
596,Tyler Conklin,TE,45.724132
265,Jake Butt,TE,46.039694
412,Mark Walton,RB,47.701821
313,John Kelly,RB,48.017383
375,KhaDarel Hodge,WR,48.392099
175,De'Lance Turner,RB,48.392099


In [27]:
freeagents = [NflPlayer(*p) for p in nfl_players.itertuples(index=False, name=None)]


In [28]:
import pandas as pd

num_competitors = 12
rosters = [[] for _ in range(num_competitors)] # empty rosters to start with

num_rounds = 10
turns = []
# generate turns by snake order
for i in range(num_rounds):
    turns += reversed(range(num_competitors)) if i % 2 else range(num_competitors)
    
state = DraftState(rosters, turns, freeagents)
iterations = 1000
while state.GetMoves() != []:
    move = UCT(state, iterations)
    state.DoMove(move)
    print(state.rosters)



  0%|          | 4/1000 [00:00<00:29, 34.04it/s]

[[Aaron Jones|RB|139.79665828900426], [], [], [], [], [], [], [], [], [], [], []]



  1%|          | 6/1000 [00:00<00:18, 53.78it/s]

[[Aaron Jones|RB|139.79665828900426], [Aaron Rodgers|QB|191.7639345778395], [], [], [], [], [], [], [], [], [], []]



  1%|          | 6/1000 [00:00<00:17, 56.38it/s]

...

[[Aaron Jones|RB|139.79665828900426, Ameer Abdullah|RB|74.11383300832348, Alex Erickson|WR|66.5276501783472, Ben Braunecker|TE|70.54214196438288, Austin Carr|WR|104.44802098024469, Brandon LaFell|WR|95.83963328111608, Brandon Marshall|WR|99.92156892303845, Chase Daniel|QB|55.92264353571274, Blake Jarwin|TE|102.37829090002991], [Aaron Rodgers|QB|191.7639345778395, Aldrick Robinson|WR|61.3657718821842, Allen Hurns|WR|87.48864071550976, Bradley Sowell|RB|82.19419252145853, Ben Koyack|TE|73.28125614099666, Charcandrick West|RB|96.35486341742002, Charles Clay|RB|73.78125240815754, Calvin Ridley|WR|109.6060342583538, Brian Parker|TE|84.60073112336141], [Andrew Luck|QB|214.58559883098118, Alex Smith|TE|148.4784055626798, Anthony Firkser|TE|73.90234066633245, Auden Tate|WR|65.40182055354478, Brandon Bolden|RB|63.01119944842719, Chad Kelly|QB|76.0636997122612, Chase Edmonds|RB|96.38999433253656, Bruce Ellington|WR|86.21166699200589, Darrel Williams|RB|44.692253767785175, Charone Peake|WR|66.198


 13%|█▎        | 130/1000 [00:00<00:00, 1291.50it/s]

[[Aaron Jones|RB|139.79665828900426, Ameer Abdullah|RB|74.11383300832348, Alex Erickson|WR|66.5276501783472, Ben Braunecker|TE|70.54214196438288, Austin Carr|WR|104.44802098024469, Brandon LaFell|WR|95.83963328111608, Brandon Marshall|WR|99.92156892303845, Chase Daniel|QB|55.92264353571274, Blake Jarwin|TE|102.37829090002991], [Aaron Rodgers|QB|191.7639345778395, Aldrick Robinson|WR|61.3657718821842, Allen Hurns|WR|87.48864071550976, Bradley Sowell|RB|82.19419252145853, Ben Koyack|TE|73.28125614099666, Charcandrick West|RB|96.35486341742002, Charles Clay|RB|73.78125240815754, Calvin Ridley|WR|109.6060342583538, Brian Parker|TE|84.60073112336141, DeAndre Washington|RB|78.43069804740793], [Andrew Luck|QB|214.58559883098118, Alex Smith|TE|148.4784055626798, Anthony Firkser|TE|73.90234066633245, Auden Tate|WR|65.40182055354478, Brandon Bolden|RB|63.01119944842719, Chad Kelly|QB|76.0636997122612, Chase Edmonds|RB|96.38999433253656, Bruce Ellington|WR|86.21166699200589, Darrel Williams|RB|44


100%|██████████| 1000/1000 [00:00<00:00, 1180.98it/s]

[[Aaron Jones|RB|139.79665828900426, Ameer Abdullah|RB|74.11383300832348, Alex Erickson|WR|66.5276501783472, Ben Braunecker|TE|70.54214196438288, Austin Carr|WR|104.44802098024469, Brandon LaFell|WR|95.83963328111608, Brandon Marshall|WR|99.92156892303845, Chase Daniel|QB|55.92264353571274, Blake Jarwin|TE|102.37829090002991, Chester Rogers|WR|85.77142155216808], [Aaron Rodgers|QB|191.7639345778395, Aldrick Robinson|WR|61.3657718821842, Allen Hurns|WR|87.48864071550976, Bradley Sowell|RB|82.19419252145853, Ben Koyack|TE|73.28125614099666, Charcandrick West|RB|96.35486341742002, Charles Clay|RB|73.78125240815754, Calvin Ridley|WR|109.6060342583538, Brian Parker|TE|84.60073112336141, DeAndre Washington|RB|78.43069804740793], [Andrew Luck|QB|214.58559883098118, Alex Smith|TE|148.4784055626798, Anthony Firkser|TE|73.90234066633245, Auden Tate|WR|65.40182055354478, Brandon Bolden|RB|63.01119944842719, Chad Kelly|QB|76.0636997122612, Chase Edmonds|RB|96.38999433253656, Bruce Ellington|WR|86.

In [ ]:
pd.DataFrame({"Team " + str(i + 1): r for i, r in enumerate(state.rosters)})


In [10]:
state.rosters

[[Aaron Rodgers|QB|191.7639345778395],
 [Andrew Luck|QB|214.58559883098118],
 [Andy Dalton|QB|212.8374760549471],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]